In [12]:
import autodiff.forward
from autodiff.forward import *

import numpy as np
from numpy.linalg import pinv
from numpy.linalg import norm

In [6]:
class VectorFunction:
    
    def __init__(self, exprlist):
        self._exprlist = exprlist.copy()
    
    def evaluation_at(self, val_dict):
        return np.array([expr.evaluation_at(val_dict) 
                        for expr in self._exprlist])
    
    def jacobian_at(self, val_dict):
        return np.array([[f.derivative_at(var, val_dict) for var in val_dict.keys()]
                         for f in self._exprlist])

$h(x, y) = (f(x,y), g(x, y)) = (\sin(x+y), \cos(x-y))$

In [7]:
x, y = Variable(), Variable()
f, g = sin(x+y), cos(x-y)
h = VectorFunction([f, g])


$h(\dfrac{\pi}{4}, \dfrac{\pi}{4}) = (\sin(\dfrac{\pi}{2}), \cos(0)) = (1, 1)$

In [8]:
h.evaluation_at({x: np.pi/6, y: np.pi/6})

array([0.8660254, 1.       ])

When $(x, y) = (\dfrac{\pi}{6}, \dfrac{\pi}{6})$:

$$
J_{h} = 
\begin{bmatrix}
\dfrac{\partial f}{\partial x} & \dfrac{\partial f}{\partial y} \\
\dfrac{\partial g}{\partial x} & \dfrac{\partial g}{\partial y} 
\end{bmatrix}
= 
\begin{bmatrix}
\cos(x+y) & \cos(x+y) \\
-\sin(x-y) & \sin(x-y)
\end{bmatrix}
= 
\begin{bmatrix}
\cos(\dfrac{\pi}{3}) & \cos(\dfrac{\pi}{3}) \\
-\sin(0) & \sin(0)
\end{bmatrix}
=
\begin{bmatrix}
0.5 & 0.5 \\
0 & 0
\end{bmatrix}
$$

In [9]:
h.jacobian_at({x: np.pi/6, y: np.pi/6})

array([[ 0.5,  0.5],
       [-0. ,  0. ]])

In [19]:
def newton_multivariable(vectorfunction, init_val_dict, tol, max_itr, verbose):
    
    itr = 1
    val_dict = init_val_dict.copy()
    
    while True:
        
        evalvector = vectorfunction.evaluation_at(val_dict)
        jacobian   = vectorfunction.jacobian_at(val_dict)
        dx = np.dot( pinv(jacobian), evalvector )
        
        for i, v in enumerate( val_dict.keys() ):
            val_dict[v] = val_dict[v] - dx[i]
        
        if verbose: print("iteration {0}, at {1}, objective function = {2}".format( \
                          itr, list(val_dict.values()), vectorfunction.evaluation_at(val_dict)))

        if norm(vectorfunction.evaluation_at(val_dict), ord=2) <= tol:
            print("Found solution after {} iterations.".format(itr))
            print("There is a root at {}.".format(list(val_dict.values())))
            break

        if itr > max_itr:
            print("Exceeded allowable max iterations without finding a root.")
            break
        
        itr += 1
        
    return val_dict

Try using Newton'w method to solve 
$$
\begin{cases}
\cos(x)+\sin(y) = 0 \\
x^2 + y^2 = 0
\end{cases}
$$

In [25]:
entry1 = cos(x) + sin(y)
entry2 = x**2 - y**2
vector = VectorFunction([entry1, entry2])
root = newton_multivariable(vector, {x: np.pi/6, y: np.pi/6}, 1e-6, 25, verbose=True)

iteration 1, at [-3.208452031970579, -3.208452031970579], objective function = [-0.93095617  0.        ]
iteration 2, at [-4.0829379871543505, -4.0829379871543505], objective function = [0.21964977 0.        ]
iteration 3, at [-3.9257142101932674, -3.9257142101932674], objective function = [-0.00180539  0.        ]
iteration 4, at [-3.926990817680748, -3.926990817680748], objective function = [9.80766024e-10 0.00000000e+00]
Found solution after 4 iterations.
There is a root at [-3.926990817680748, -3.926990817680748].


Verify that this is indeed a root.

In [26]:
np.cos(root[x]) + np.sin(root[y])

9.807660239502525e-10

In [27]:
root[x]**2 - root[y]**2

0.0